# RealSenseを使ってみよう（平面の自動検出）
### RealSenseを接続してください。

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
from IPython.display import Image, display
import matplotlib.pyplot as plt
%matplotlib inline

# Configure color and depth to run at VGA resolution at 30 frames per second
config = rs.config()
config.enable_stream(rs.stream.depth)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

## 計算の準備しよう。
"?????" を正しい計算式に書き換えてから実行してください。乗算は *，除算は / です。
- X = ?????   # Z, u, f による計算式
- Y = ?????   # Z, v, f による計算式


In [ ]:
focal_length = intr.fx # in pixels

def Zuv_to_XY(Z, u, v, f=focal_length):
    X = ?????   # Z, u, f による計算式
    Y = ?????   # Z, v, f による計算式
    return X, Y

In [ ]:
# 平面を検出するRANSACの実装

def DetectPlane(points, n_trials=30, th=3):

    # initial settings
    plane = dict(normal=None, p3idx=None)
    n_max, dev_min = 0, float("inf")

    for i in range(n_trials):
        # randomly pick up three points
        p3idx = np.random.choice(points.shape[0], 3, replace=False)
        # compute a unit normal vector
        normal = np.cross(points[p3idx[1]] - points[p3idx[0]], points[p3idx[2]] - points[p3idx[0]])
        normal = normal / np.linalg.norm(normal)

        # compute distances from the plane with a point p3idx[0] and the normal vector
        distances = np.abs(np.dot(points - points[p3idx[0],:], normal))
        
        # find the neighboring points to the plane
        pidx_neighbors = np.where(distances < th)[0]
        num_neighbors = len(pidx_neighbors)
        deviation = np.std(distances[pidx_neighbors])

        # check if the plane is better than the current estimate
        if num_neighbors > n_max or (num_neighbors == n_max and deviation < dev_min):
            n_max, dev_min = num_neighbors, deviation
            plane["normal"], plane["p3idx"] = normal, p3idx

    return plane

In [ ]:
# 設定項目（この初期設定で大丈夫です）
disp_mm = 20.           # [mm]　この距離以内の点を平面と見なして着色する。
max_depth = 2.0 / scale # [m]　この距離より遠い点は計算の対象にしない。

n_trials = 300          # [回]　この回数だけRANSACの手順を繰り返す度に，結果を表示する。
th = 10                 # [mm]　RANSACでは，この距離より近い点を平面にとても近いと見なして数える。
n = 10000               # [個]　全点ではなく，ランダムに間引く。この個数の点が計算に使われる。
mask_color = np.array((0, 0, 128),dtype=np.uint8)

以上で準備完了です。

## 平面の自動検出を実行しよう。
- 平面として検出された領域が着色されて表示されます。
- スペースキーを押す毎に画像が連番で保存されます。'q' を押すと終了します。

In [ ]:
# 平面の自動検出

pipeline = rs.pipeline()
profile = pipeline.start(config)

# Get camera parameters
intr = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
scale = config.resolve(rs.pipeline_wrapper(pipeline)).get_device().first_depth_sensor().get_depth_scale()
cx, cy = intr.ppx, intr.ppy # width*0.5, height*0.5
height = intr.height
width = intr.width
j_to_u = lambda j: -(j - cx)
i_to_v = lambda i: -(i - cy)
u, v = np.meshgrid(j_to_u(np.arange(width)), i_to_v(np.arange(height)))

# Create a camera alignment object (depth aligned to color)
align = rs.align(rs.stream.color)
print("Press [SPACE] to save images (png) and depth data (npy).")
print("Press 'q' to stop.")
nsaved = 0
try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.get_color_frame()
        depth_frame = aligned_frames.get_depth_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        bgr = np.asanyarray(color_frame.get_data())
        depth = np.asanyarray(depth_frame.get_data())
        depth[depth > max_depth] = 0 # Zeros out
        
        Z = depth * scale * 1e+3 # unit in mm
        X, Y = Zuv_to_XY(Z, u, v, intr.fx)
        points = np.vstack((X[Z>0],Y[Z>0],Z[Z>0])).T
        nd = np.count_nonzero(Z)
        p = np.random.choice(nd, min(n,nd), replace=False)
        points = points[p]
        plane = DetectPlane(points, n_trials=n_trials, th=th)
        tmp = np.vstack((X.flatten(),Y.flatten(),Z.flatten())).T
        distances = np.abs(np.dot(tmp - points[plane['p3idx'][0],:], plane['normal']))
        
        distances = distances.reshape(480,640)
        mask = np.zeros(480* 640* 3).reshape(480, 640, 3)
        mask[distances < disp_mm] = mask_color
        bgr = bgr.astype(np.uint32) + mask.astype(np.uint32)
        bgr[bgr > 255] = 255
        bgr = bgr.astype(np.uint8)
        
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U), 
                                           cv2.COLORMAP_JET)
        
        images = np.hstack((bgr, depth_colormap))
        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        
        key = cv2.waitKey(100)
        if key == ord(' '):
            Z = depth * scale * 1e+3 # unit in mm
            color = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            
            # Save images
            cv2.imwrite('color{:02d}pc.png'.format(nsaved), bgr)
            cv2.imwrite('depth{:02d}pc.png'.format(nsaved), depth_colormap)
            np.save('Z{:02d}pc.npy'.format(nsaved), Z)
            
            print("color image and depth data are saved ({:02d})".format(nsaved))
            nsaved += 1

        if key == ord('q'):
            if nsaved == 0:
                Z = depth * scale * 1e+3 # unit in mm
                color = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            cv2.destroyAllWindows()
            break
        
finally:
    # Stop streaming
    pipeline.stop()